In [ ]:
### All code stolen from https://blinkletter.github.io/PythonPresentation/03C-Eyring_Exercises_3_LMfit.html and adapted.


### Setup environment

!mkdir plots

### Install and load packages
 
!pip install uncertainties     # uncomment to install dependancy
!pip install lmfit             # for docs see https://lmfit.github.io/lmfit-py/


import lmfit
import numpy as np                    # import the tools of NumPy as "np"
from matplotlib import pyplot as plt  # tools for plotting
import scipy.constants as const  # physical constants

import uncertainties as un               # tool set for uncertain numbers
from uncertainties import unumpy as unp  # a replacement for numpy

### Set global variables

#location_data = "data/"         ## Use either local folder or github folder. 
#location_styles = "styles/"     ## Use github locations for Colab
location_data = "https://raw.githubusercontent.com/blinkletter/PythonPresentation/main/data/"
location_styles = "https://raw.githubusercontent.com/blinkletter/PythonPresentation/main/styles/"

plt.rcdefaults()

size = [4,4]
size2 = [4,5]
size3 = [2.5,2.5]

In [60]:
######################
### Experimental data with error
######################

#temp = [293, 298, 303, 308, 313]       # list of temperatures
#k_obs = [7.6, 11.7, 15.2, 21.3, 27.8]  # list of observe rate constants (s^-1)
#k_obs_err= [0.2 , 0.3, 0.1, 0.9, 0.9]  # list of standard deviations for data


time = [0, 0.042, 0.092, 0.119, 0.151, 0.280, 0.435]     # /min
conc = [0.250, 0.213, 0.175, 0.156, 0.138, 0.081, 0.044]  # /M
conc_err = [0.004, 0.004, 0.003, 0.003, 0.003, 0.004, 0.003]  # /M

#time = [0, 0.042, 0.092, 0.119, 0.151, 0.280, 0.435]     # /min
#conc = [0.250, 0.213, 0.185, 0.156, 0.128, 0.081, 0.044]  # /M
#conc_err = [0.004, 0.004, 0.003, 0.003, 0.003, 0.004, 0.003]  # /M


### Convert lists to numpy arrays (enables numpy math tools with these lists)
time = np.array(time)
conc_u = unp.uarray(conc, conc_err)   # make an array of ufloat values



In [ ]:
######################
###  Linear function to be used by curve_fit
######################
def linear(x, m, b):
    y = m * x + b
    return y


### Set up data

### Calculations for First Order plot axes
x = time * 60  # x is now an array of floats (seconds)
y_u = unp.log(conc_u)   # y_u is now an array of ufloats

y = unp.nominal_values(y_u) # extract arrays of nominal values and errors
y_err = unp.std_devs(y_u)   # because curve_fit can handle ufloats


### Use curve_fit function  

# load the function f as the model to be fit
mod = lmfit.Model(linear)       

# state the parameters (use the text strings that we used above in the 
#   function) initial values are also set here
pars = mod.make_params(m=-1000, b=-1  )     
                                            
# use the .fit method on the model object to perform the curve fit
result = mod.fit(y, pars, x=x, 
                 weights=1.0/y_err
                 )   

print("---------- FIT REPORT ------------")
print(result.fit_report())

r,p = scipy.stats.pearsonr(x,y); rsq = r**2
print(f"RSQ = {rsq:0.3f}")
print()

#print("--- PARAMETER CONFIDENCE INTERVALS ----")
print(result.ci_report())
#print()

intercept = result.uvars['b']      # collect parameters as uncertain values 
slope = result.uvars['m']

k_u = -slope
A0 = unp.exp(intercept)

print("\n")
print("------THERMODYNAMIC PARAMETERS----------")
print(f"The value of k is {k_u:0.5f} /sec")
print(f"The initial [A] is {A0:0.5f} M")
print()



In [ ]:
######################
### Plot the results
######################

plt.rcdefaults()
style = "tufte.mplstyle"
#style = "S2_classic2.mplstyle"
style_name = location_styles + style
plt.style.use(style_name)

intercept = result.uvars['b']      # collect parameters as uncertain values 
slope = result.uvars['m']

fig = plt.figure(figsize = size2)
result.plot(fig = fig,
            fig_kws = {},      # this is a keyword dictionary for figure styles
            xlabel = r"$t\text{ /s}$",
            ylabel = r"$\ln \left(\left[ \text{A} \right]\text{ /M}\right)$",
            yerr = 2*y_err,
            title = " ",
            data_kws ={"color":"black", "linestyle":"None", "linewidth": 0.5,
                       "markerfacecolor": "white", "markeredgecolor":"black", 
                       "markeredgewidth":0.5, "zorder":3},
            fit_kws ={"linewidth": 0.5, "zorder":2},
            ax_res_kws = {},   # 'kws' is 'keyword styles'
            ax_fit_kws = {},
            )
 
ax = fig.axes   # get the axes from the plot (there are two in a list)

#ax[0].set_xticks([])
#ax[1].set_xticks([0,10,20,30]) # These will be the ticks for both axes


### Confidence band
sigma = 2

x_0 = np.linspace(np.min(x), np.max(x), 100)
y_0 = result.eval(x=x_0)
dely = result.eval_uncertainty(x=x)
dely_0 = result.eval_uncertainty(x=x_0, sigma=sigma)

ax[1].fill_between(x_0, y_0-dely_0, y_0+dely_0, 
                   linewidth=0, color='lightgray', alpha = 0.4, zorder=1
                   )
ax[0].fill_between(x_0, -dely_0, dely_0, 
                   linewidth=0, color='lightgray', alpha = 0.4, zorder=1
                   )

### do not show legend 
#fig.legend([])
ax[1].legend([])      # blank legend

plt.savefig("plots/plot1.pdf")
plt.show()


In [ ]:
######## SECOND ORDER

### Calculations for Second Order plot axes
x = time * 60  # x is now an array of floats (seconds)
y_u = 1/(conc_u)   # y_u is now an array of ufloats

y = unp.nominal_values(y_u) # extract arrays of nominal values and errors
y_err = unp.std_devs(y_u)   # because curve_fit can handle ufloats


######################
###  Linear function to be used by curve_fit
######################
def linear(x, m, b):
    y = m * x + b
    return y

### Use curve_fit function  

# load the function f as the model to be fit
mod = lmfit.Model(linear)       

# state the parameters (use the text strings that we used above in the 
#   function) initial values are also set here
pars = mod.make_params(m=-1000, b=-1  )     
                                            
# use the .fit method on the model object to perform the curve fit
result = mod.fit(y, pars, x=x, 
#                 weights=1.0/y_err
                 )   

print("---------- FIT REPORT ------------")
print(result.fit_report())

r,p = scipy.stats.pearsonr(x,y); rsq = r**2
print(f"RSQ = {rsq:0.3f}")
print()

#print("--- PARAMETER CONFIDENCE INTERVALS ----")
print(result.ci_report())
#print()

intercept = result.uvars['b']      # collect parameters as uncertain values 
slope = result.uvars['m']

k_u = -slope
A0 = unp.exp(intercept)

print("\n")
print("------THERMODYNAMIC PARAMETERS----------")
print(f"The value of k is {k_u:0.5f} /sec")
print(f"The initial [A] is {A0:0.5f} M")
print()


######################
### Plot the results
######################

plt.rcdefaults()
style = "tufte.mplstyle"
#style = "S2_classic2.mplstyle"
style_name = location_styles + style
plt.style.use(style_name)

intercept = result.uvars['b']      # collect parameters as uncertain values 
slope = result.uvars['m']

fig = plt.figure(figsize = size2)
result.plot(fig = fig,
            fig_kws = {},      # this is a keyword dictionary for figure styles
            xlabel = r"$t\text{ /s}$",
            ylabel = r"$1/ \left(\left[ \text{A} \right]\text{ /M}\right)$",
            yerr = 2*y_err,
            title = " ",
            data_kws ={"color":"black", "linestyle":"None", "linewidth": 0.5,
                       "markerfacecolor": "white", "markeredgecolor":"black", 
                       "markeredgewidth":0.5, "zorder":3},
            fit_kws ={"linewidth": 0.5, "zorder":2},
            ax_res_kws = {},   # 'kws' is 'keyword styles'
            ax_fit_kws = {},
            )
 
ax = fig.axes   # get the axes from the plot (there are two in a list)

#ax[0].set_xticks([])
#ax[1].set_xticks([0,10,20,30]) # These will be the ticks for both axes


### Confidence band
sigma = 2

x_0 = np.linspace(np.min(x), np.max(x), 100)
y_0 = result.eval(x=x_0)
dely = result.eval_uncertainty(x=x)
dely_0 = result.eval_uncertainty(x=x_0, sigma=sigma)

ax[1].fill_between(x_0, y_0-dely_0, y_0+dely_0, 
                   linewidth=0, color='lightgray', alpha = 0.4, zorder=1
                   )
ax[0].fill_between(x_0, -dely_0, dely_0, 
                   linewidth=0, color='lightgray', alpha = 0.4, zorder=1
                   )
#ax[0].set_ylim([-5,5]) ### ADDED this to set residual scale

### do not show legend 
#fig.legend([])
ax[1].legend([])      # blank legend

plt.savefig("plots/plot2.pdf")
plt.show()


In [ ]:
######## FIRST ORDER Non-Linear

### Calculations for Second Order plot axes
x = time * 60  # x is now an array of floats (seconds)
y_u = (conc_u)   # y_u is now an array of ufloats

y = unp.nominal_values(y_u) # extract arrays of nominal values and errors
y_err = unp.std_devs(y_u)   # because curve_fit can handle ufloats


######################
###  integrated function to be used by curve_fit
######################
def first_order(x, k, A0):
    y = A0 * np.exp(-k*x)
    return y

### Use curve_fit function  

# load the function f as the model to be fit
mod = lmfit.Model(first_order)       

# state the parameters (use the text strings that we used above in the 
#   function) initial values are also set here
pars = mod.make_params(k=0.1, A0=-1  )     
                                            
# use the .fit method on the model object to perform the curve fit
result = mod.fit(y, pars, x=x, 
                 weights=1.0/y_err
                 )   

print("---------- FIT REPORT ------------")
print(result.fit_report())

r,p = scipy.stats.pearsonr(x,y); rsq = r**2
print(f"RSQ = {rsq:0.3f}")
print()

#print("--- PARAMETER CONFIDENCE INTERVALS ----")
print(result.ci_report())
#print()

k_u = result.uvars['k']      # collect parameters as uncertain values 
A0_u = result.uvars['A0']


print("\n")
print("------THERMODYNAMIC PARAMETERS----------")
print(f"The value of k is {k_u:0.5f} /sec")
print(f"The initial [A] is {A0_u:0.5f} M")
print()


######################
### Plot the results
######################

plt.rcdefaults()
style = "tufte.mplstyle"
#style = "S2_classic2.mplstyle"
style_name = location_styles + style
plt.style.use(style_name)

k_u = result.uvars['k']      # collect parameters as uncertain values 
A0_u = result.uvars['A0']

fig = plt.figure(figsize = size2)
result.plot(fig = fig,
            fig_kws = {},      # this is a keyword dictionary for figure styles
            xlabel = r"$t\text{ /s}$",
            ylabel = r"$\left[ \text{A} \right]\text{ /M}$",
            yerr = 2*y_err,
            title = " ",
            data_kws ={"color":"black", "linestyle":"None", "linewidth": 0.5,
                       "markerfacecolor": "white", "markeredgecolor":"black", 
                       "markeredgewidth":0.5, "zorder":3},
            fit_kws ={"linewidth": 0.5, "zorder":2},
            ax_res_kws = {},   # 'kws' is 'keyword styles'
            ax_fit_kws = {},
            numpoints=1000  #### added numpoints to get smooth curves
            )
 
ax = fig.axes   # get the axes from the plot (there are two in a list)

#ax[0].set_xticks([])
#ax[1].set_xticks([0,10,20,30]) # These will be the ticks for both axes


### Confidence band
sigma = 2

x_0 = np.linspace(np.min(x), np.max(x), 100)
y_0 = result.eval(x=x_0)
dely = result.eval_uncertainty(x=x)
dely_0 = result.eval_uncertainty(x=x_0, sigma=sigma)

ax[1].fill_between(x_0, y_0-dely_0, y_0+dely_0, 
                   linewidth=0, color='lightgray', alpha = 0.4, zorder=1
                   )
ax[0].fill_between(x_0, -dely_0, dely_0, 
                   linewidth=0, color='lightgray', alpha = 0.4, zorder=1
                   )
ax[0].set_ylim([-0.025,0.025]) ### ADDED this to set residual scale

### do not show legend 
#fig.legend([])
ax[1].legend([])      # blank legend

plt.savefig("plots/plot3.pdf")
plt.show()


In [ ]:
######## SECOND ORDER Non-Linear

### Calculations for Second Order plot axes
x = time * 60  # x is now an array of floats (seconds)
y_u = (conc_u)   # y_u is now an array of ufloats

y = unp.nominal_values(y_u) # extract arrays of nominal values and errors
y_err = unp.std_devs(y_u)   # because curve_fit can handle ufloats


######################
###  integrated function to be used by curve_fit
######################
def second_order(x, k, A0):
    y = A0/(1+A0*k*x)
    return y

### Use curve_fit function  

# load the function f as the model to be fit
mod = lmfit.Model(second_order)       

# state the parameters (use the text strings that we used above in the 
#   function) initial values are also set here
pars = mod.make_params(k=0.1, A0=-1  )     
                                            
# use the .fit method on the model object to perform the curve fit
result = mod.fit(y, pars, x=x, 
                 weights=1.0/y_err
                 )   

print("---------- FIT REPORT ------------")
print(result.fit_report())

r,p = scipy.stats.pearsonr(x,y); rsq = r**2
print(f"RSQ = {rsq:0.3f}")
print()

#print("--- PARAMETER CONFIDENCE INTERVALS ----")
print(result.ci_report())
#print()

k_u = result.uvars['k']      # collect parameters as uncertain values 
A0_u = result.uvars['A0']


print("\n")
print("------THERMODYNAMIC PARAMETERS----------")
print(f"The value of k is {k_u:0.5f} /sec")
print(f"The initial [A] is {A0_u:0.5f} M")
print()


######################
### Plot the results
######################

plt.rcdefaults()
style = "tufte.mplstyle"
#style = "S2_classic2.mplstyle"
style_name = location_styles + style
plt.style.use(style_name)

k_u = result.uvars['k']      # collect parameters as uncertain values 
A0_u = result.uvars['A0']

fig = plt.figure(figsize = size2)
result.plot(fig = fig,
            fig_kws = {},      # this is a keyword dictionary for figure styles
            xlabel = r"$t\text{ /s}$",
            ylabel = r"$\left[ \text{A} \right]\text{ /M}$",
            yerr = 2*y_err,
            title = " ",
            data_kws ={"color":"black", "linestyle":"None", "linewidth": 0.5,
                       "markerfacecolor": "white", "markeredgecolor":"black", 
                       "markeredgewidth":0.5, "zorder":3},
            fit_kws ={"linewidth": 0.5, "zorder":2},
            ax_res_kws = {},   # 'kws' is 'keyword styles'
            ax_fit_kws = {},
            numpoints=1000  #### added numpoints to get smooth curves
            )
 
ax = fig.axes   # get the axes from the plot (there are two in a list)

#ax[0].set_xticks([])
#ax[1].set_xticks([0,10,20,30]) # These will be the ticks for both axes


### Confidence band
sigma = 2

x_0 = np.linspace(np.min(x), np.max(x), 100)
y_0 = result.eval(x=x_0)
dely = result.eval_uncertainty(x=x)
dely_0 = result.eval_uncertainty(x=x_0, sigma=sigma)

ax[1].fill_between(x_0, y_0-dely_0, y_0+dely_0, 
                   linewidth=0, color='lightgray', alpha = 0.4, zorder=1
                   )
ax[0].fill_between(x_0, -dely_0, dely_0, 
                   linewidth=0, color='lightgray', alpha = 0.4, zorder=1
                   )

### do not show legend 
#fig.legend([])
ax[1].legend([])      # blank legend

plt.savefig("plots/plot4.pdf")
plt.show()
